# Test Documentation

## Parameters to test:

- prompt (scaling)
- temperature and top_p
- model (gpt-3.5-turbo-0125 vs gpt-4-turbo-2024-04-09)
- anonymized data vs non-anonymized data

prompts: 

1.prompt = f"Consider the sentiment of this article towards {bank_name}. If {bank_name} is not mentioned, assign a neutral score of 0.5. Otherwise, score the sentiment from 0 to 1, where 0 is very negative and 1 is very positive. Provide the sentiment score followed by a brief explanation in less than 20 words, separating them with a period. Example: '[insert sentiment score]. [insert explanation]'."


    - scaling from 0 to 1 continuous in 0.1 steps

2.prompt = f"Consider the sentiment of this article towards {bank_name}. Score the sentiment from -1 to 1, where -1 is very negative and 1 is very positive. Provide the sentiment score followed by a brief explanation in less than 20 words, separating them with a period. Example: '[insert sentiment score]. [insert explanation]'."


    - scaling from -1 to 1 continuous in 0.1 steps

3.prompt = f"Analyze the sentiment of this article towards {bank_name}. Score the sentiment as either 1 for positive, -1 for negative, or 0 for neutral. Provide the sentiment score (1,0, or -1), followed by a brief explanation in less than 20 words, separating them with a period. Example: '[insert sentiment score]. [insert explanation]'."


    - discrete scaling 1, 0, -1

## Information regarding prompt design:
Didnt really document the different tests that involved creating the prompts since the output seemed pretty random in relation to what i changed for the prompts. Tested it on a smaller dataset with 10 articles. Tried to find the best configuration to help with quality and consistency. Here are some observations:
- "Example: '[insert sentiment score]. [insert explanation]'". was added to help with consistency of the model output. Wasnt really needed in the first prompt, but also didnt influence the output in a negative way. Helped with the consistency of the output for the second and third prompt, however it also seemed to change the scores of the output somewhat. 
- "If {bank_name} is not mentioned, assign a neutral score of 0.5." was added to first prompt since it helped with the quality of the output. It didnt skew the output towards neutral. This was different for the second and third prompt where the output was heavily skewed towards neutral when adding "If {bank_name} is not mentioned, assign a neutral score of 0."
- GPT seems pretty random and i dont really understand why inserting an example of how the output should look like would influence its answers in regard to the score. Also dont understand why adding "If {bank_name} is not mentioned, assign a neutral score of 0." would skew the output towards neutral in 1 prompt but not in the other. 
- For continuous scaling the terms very positive and very negative were used (made sense), but not for discrete scaling.
- Also tried to use words like "positive", "negative" and "neutral" in the prompt instead of 1, 0, -1 to see if it would help with the consistency of the output. It didnt, the output quality was worse and it didnt help consistency.


## Sentiment score labels for dataset (determined by reading the article): 

Article 0: Some positive sentiment (returns up), CS generally in a difficult position. 

Article 1: Very negative (Archegos stuff)

Article 2: Neutral – CS providing insights 

Article 3: Negative – spying scandal

Article 4: Eher negative (AT1 bonds after takeover)

Article 5: Eher negative (article is mainly about deutsche bank not being the next credit Suisse)

Article 6: (Neutral,) slightly positive (CS helping Climate bonds initiative)

Article 7: Negative – lawsuits blabla

Article 8: Neutral to slightly negative. (princeling hiring practice in banking sector)

Article 9: Negative (credit Suisse takeover, restructuring blabla…) 

Article 10: More or less neutral, negativ towards previous management, optimistic about new management. 

Article 11: Eher negativ

Article 12: Eher negativ

Article 13: Negativ.

Article 14: Neutral to slightly positiv. 

Article 15: Negativ 

Article 16: Slightly negativ to neutral

Article 17: Negativ

Article 18: Optimistic, probably neutral to slightly positiv

Article 19: Handelt von der PUK – neutral to slightly negativ



In [1]:
import pandas as pd
import re
import numpy as np

Test R1-1-1 to R1-1-5
Parameters: 

- prompt:
    - 1.prompt = f"Consider the sentiment of this article towards {bank_name}. If {bank_name} is not mentioned, assign a neutral score of 0.5. Otherwise, score the sentiment from 0 to 1, where 0 is very negative and 1 is very positive. Provide the sentiment score followed by a brief explanation in less than 20 words, separating them with a period. Example: '[insert sentiment score]. [insert explanation]'." 
      - scaling from 0 to 1 continuous in 0.1 steps
     
- temperature = 0
- top_p = 0
- model = gpt-3.5-turbo-0125
- non-anonymized data

In [4]:

# Assuming the files are named run1.csv, run2.csv, etc.
file_names_11 = ['Output_Testdata/CS/sample_data_20_20240512_150253_R1-1-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_151052_R1-1-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_151211_R1-1-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_151320_R1-1-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_151411_R1-1-5.csv']
dataframes_11 = [pd.read_csv(file_name) for file_name in file_names_11]

In [6]:
def extract_score(sentiment_text):
    match = re.match(r"(-?\d\.?\d*)\.", sentiment_text)
    return float(match.group(1)) if match else None

for df_11 in dataframes_11:
    df_11['Extracted_Score'] = df_11['Sentiment'].apply(extract_score)

In [7]:
# Create a DataFrame to hold the extracted scores from all runs
scores_11 = pd.DataFrame({
    f'Run{i+1}': df_11['Extracted_Score'] for i, df_11 in enumerate(dataframes_11)
})

# Calculate mean and standard deviation across runs for each article
scores_11['Mean'] = scores_11.mean(axis=1)
#scores['StdDev'] = scores.std(axis=1)

# Function to calculate the percentage of identical values only considering the first 5 columns (run results)
def identical_percentage(row):
    # Isolate the first five columns which are the run results
    data = row[:5]
    unique_values = set(data.dropna())  # Get unique values excluding NaN
    if len(unique_values) == 1:
        return 100.0  # All values are identical
    else:
        # Count occurrences of the most common value
        most_common = max(set(data), key=list(data).count)
        count_most_common = list(data).count(most_common)
        return (count_most_common / len(data.dropna())) * 100

# Apply the function to each row and store the results in a new column
scores_11['Identical Score Percentage'] = scores_11.iloc[:, :5].apply(identical_percentage, axis=1)

# Show the first 20 rows to inspect mean, standard deviation, and identical score percentage

print(scores_11.head(20))

    Run1  Run2  Run3  Run4  Run5  Mean  Identical Score Percentage
0    0.7   0.7   0.7   0.7   0.7  0.70                       100.0
1    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
2    0.7   0.7   0.7   0.7   0.7  0.70                       100.0
3    0.7   0.7   0.6   0.6   0.7  0.66                        60.0
4    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
5    0.5   0.5   0.5   0.5   0.5  0.50                       100.0
6    0.8   0.8   0.8   0.8   0.8  0.80                       100.0
7    0.3   0.3   0.3   0.3   0.3  0.30                       100.0
8    0.6   0.6   0.6   0.6   0.6  0.60                       100.0
9    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
10   0.7   0.7   0.7   0.7   0.7  0.70                       100.0
11   0.3   0.3   0.3   0.3   0.3  0.30                       100.0
12   0.3   0.3   0.3   0.3   0.3  0.30                       100.0
13   0.2   0.2   0.2   0.2   0.2  0.20                       1

In [5]:
from scipy.stats import f_oneway

# Perform one-way ANOVA across runs
f_val, p_val = f_oneway(scores['Run1'], scores['Run2'], scores['Run3'], scores['Run4'], scores['Run5'])
print("ANOVA results: F-Value =", f_val, "P-Value =", p_val)

ANOVA results: F-Value = 0.0048979170963085236 P-Value = 0.9999513428275769


Test R1-2-1 to R1-2-5
Parameters:

prompt = f"Consider the sentiment of this article towards {bank_name}. If {bank_name} is not mentioned, assign a neutral score of 0.5. Otherwise, score the sentiment from 0 to 1, where 0 is very negative and 1 is very positive. Provide the sentiment score followed by a brief explanation in less than 20 words, separating them with a period. Example: '[insert sentiment score]. [insert explanation]'."

     - scaling from 0 to 1 continuous in 0.1 steps
rest same

In [11]:

# Assuming the files are named run1.csv, run2.csv, etc.
file_names = ['Output_Testdata/CS/sample_data_20_20240512_151608_R1-2-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_151757_R1-2-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_152013_R1-2-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_152123_R1-2-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_152246_R1-2-5.csv']
dataframes = [pd.read_csv(file_name) for file_name in file_names]

In [12]:
def extract_score(sentiment_text):
    match = re.match(r"(-?\d\.?\d*)\.", sentiment_text)
    return float(match.group(1)) if match else None

for df in dataframes:
    df['Extracted_Score'] = df['Sentiment'].apply(extract_score)

In [13]:
# Create a DataFrame to hold the extracted scores from all runs
scores = pd.DataFrame({
    f'Run{i+1}': df['Extracted_Score'] for i, df in enumerate(dataframes)
})

# Calculate mean and standard deviation across runs for each article
scores['Mean'] = scores.mean(axis=1)
#scores['StdDev'] = scores.std(axis=1)

# Function to calculate the percentage of identical values only considering the first 5 columns (run results)
def identical_percentage(row):
    # Isolate the first five columns which are the run results
    data = row[:5]
    unique_values = set(data.dropna())  # Get unique values excluding NaN
    if len(unique_values) == 1:
        return 100.0  # All values are identical
    else:
        # Count occurrences of the most common value
        most_common = max(set(data), key=list(data).count)
        count_most_common = list(data).count(most_common)
        return (count_most_common / len(data.dropna())) * 100

# Apply the function to each row and store the results in a new column
scores['Identical Score Percentage'] = scores.iloc[:, :5].apply(identical_percentage, axis=1)

# Show the first 20 rows to inspect mean, standard deviation, and identical score percentage
print(scores.head(20))

    Run1  Run2  Run3  Run4  Run5  Mean  Identical Score Percentage
0   -0.2  -0.5   0.5  -0.5   0.5 -0.04                        40.0
1   -0.8  -0.8  -0.8  -0.8  -0.8 -0.80                       100.0
2   -0.6  -0.6  -0.6  -0.5  -0.6 -0.58                        80.0
3   -0.5  -0.5  -0.5  -0.5  -0.5 -0.50                       100.0
4   -0.8  -0.8  -0.8  -0.8  -0.8 -0.80                       100.0
5   -0.5  -0.5  -0.5  -0.5  -0.5 -0.50                       100.0
6    0.7   0.7   0.6   0.7   0.6  0.66                        60.0
7   -0.7  -0.7  -0.7  -0.7  -0.7 -0.70                       100.0
8   -0.5  -0.5  -0.6  -0.5  -0.5 -0.52                        80.0
9   -0.8  -0.8  -0.8  -0.7  -0.8 -0.78                        80.0
10   0.6   0.6   0.6   0.6   0.6  0.60                       100.0
11  -0.7  -0.7  -0.7  -0.7  -0.7 -0.70                       100.0
12  -0.7  -0.7  -0.7  -0.7  -0.6 -0.68                        80.0
13  -0.8  -0.8  -0.8  -0.8  -0.8 -0.80                       1

Test R1-3-1 to R1-3-5

Parameters:

  3.prompt = f"Analyze the sentiment of this article towards {bank_name}. Score the sentiment as either 1 for positive, -1 for negative, or 0 for neutral. Provide the sentiment score (1,0, or -1), followed by a brief explanation in less than 20 words, separating them with a period. Example: '[insert sentiment score]. [insert explanation]'."
  
    - discrete scaling 1, 0, -1
    
rest same

In [14]:

# Assuming the files are named run1.csv, run2.csv, etc.
file_names = ['Output_Testdata/CS/sample_data_20_20240512_152454_R1-3-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_152611_R1-3-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_152713_R1-3-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_152838_R1-3-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_152912_R1-3-5.csv']
dataframes = [pd.read_csv(file_name) for file_name in file_names]

In [15]:
def extract_score(sentiment_text):
    match = re.match(r"(-?\d\.?\d*)\.", sentiment_text)
    return float(match.group(1)) if match else None

for df in dataframes:
    df['Extracted_Score'] = df['Sentiment'].apply(extract_score)

In [16]:
# Create a DataFrame to hold the extracted scores from all runs
scores = pd.DataFrame({
    f'Run{i+1}': df['Extracted_Score'] for i, df in enumerate(dataframes)
})

# Calculate mean and standard deviation across runs for each article
scores['Mean'] = scores.mean(axis=1)
#scores['StdDev'] = scores.std(axis=1)

# Function to calculate the percentage of identical values only considering the first 5 columns (run results)
def identical_percentage(row):
    # Isolate the first five columns which are the run results
    data = row[:5]
    unique_values = set(data.dropna())  # Get unique values excluding NaN
    if len(unique_values) == 1:
        return 100.0  # All values are identical
    else:
        # Count occurrences of the most common value
        most_common = max(set(data), key=list(data).count)
        count_most_common = list(data).count(most_common)
        return (count_most_common / len(data.dropna())) * 100

# Apply the function to each row and store the results in a new column
scores['Identical Score Percentage'] = scores.iloc[:, :5].apply(identical_percentage, axis=1)

# Show the first 20 rows to inspect mean, standard deviation, and identical score percentage
print(scores.head(20))

    Run1  Run2  Run3  Run4  Run5  Mean  Identical Score Percentage
0   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
1   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
2   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
3   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
4   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
5   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
6    0.0   0.0   0.0   0.0   0.0   0.0                       100.0
7   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
8   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
9   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
10  -1.0   0.0   0.0   0.0   0.0  -0.2                        80.0
11  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
12  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
13  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       1

R1-1-1 to R1-1-5 (1. prompt) is the most consistent and R1-3-1 to R1-3-5 (3. prompt) is the second most consistent. The second prompt seems to struggle with consistency as well as output formating. 


Next i test the 1. and 3. prompt with gpt 4 turbo. 



R2-1-1 to R2-1-5 

Parameters:
- 1.prompt = f"Consider the sentiment of this article towards {bank_name}. If {bank_name} is not mentioned, assign a neutral score of 0.5. Otherwise, score the sentiment from 0 to 1, where 0 is very negative and 1 is very positive. Provide the sentiment score followed by a brief explanation in less than 20 words, separating them with a period. Example: '[insert sentiment score]. [insert explanation]'."
    - scaling from 0 to 1 continuous in 0.1 steps

- temperature = 0, top_p = 0 
- model = gpt-4-turbo-2024-04-09
- non-anonymized data


In [8]:

# Assuming the files are named run1.csv, run2.csv, etc.
file_names_21 = ['Output_Testdata/CS/sample_data_20_20240512_175004_R2-1-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_175149_R2-1-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_175346_R2-1-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_175529_R2-1-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_175635_R2-1-5.csv']
dataframes_21 = [pd.read_csv(file_name) for file_name in file_names_21]

In [9]:
def extract_score(sentiment_text):
    match = re.match(r"(-?\d\.?\d*)\.", sentiment_text)
    return float(match.group(1)) if match else None

for df_21 in dataframes_21:
    df_21['Extracted_Score'] = df_21['Sentiment'].apply(extract_score)

In [10]:
# Create a DataFrame to hold the extracted scores from all runs
scores_21 = pd.DataFrame({
    f'Run{i+1}': df_21['Extracted_Score'] for i, df_21 in enumerate(dataframes_21)
})

# Calculate mean and standard deviation across runs for each article
scores_21['Mean'] = scores_21.mean(axis=1)
#scores['StdDev'] = scores.std(axis=1)

# Function to calculate the percentage of identical values only considering the first 5 columns (run results)
def identical_percentage(row):
    # Isolate the first five columns which are the run results
    data = row[:5]
    unique_values = set(data.dropna())  # Get unique values excluding NaN
    if len(unique_values) == 1:
        return 100.0  # All values are identical
    else:
        # Count occurrences of the most common value
        most_common = max(set(data), key=list(data).count)
        count_most_common = list(data).count(most_common)
        return (count_most_common / len(data.dropna())) * 100

# Apply the function to each row and store the results in a new column
scores_21['Identical Score Percentage'] = scores_21.iloc[:, :5].apply(identical_percentage, axis=1)

# Show the first 20 rows to inspect mean, standard deviation, and identical score percentage
print(scores_21.head(20))

    Run1  Run2  Run3  Run4  Run5  Mean  Identical Score Percentage
0    0.7   0.7   0.7   0.7   0.7  0.70                       100.0
1    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
2    0.7   0.7   0.7   0.7   0.7  0.70                       100.0
3    0.4   0.4   0.4   0.4   0.4  0.40                       100.0
4    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
5    0.1   0.2   0.2   0.1   0.1  0.14                        60.0
6    0.8   0.8   0.8   0.8   0.8  0.80                       100.0
7    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
8    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
9    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
10   0.4   0.4   0.4   0.4   0.4  0.40                       100.0
11   0.2   0.2   0.2   0.2   0.2  0.20                       100.0
12   0.3   0.3   0.3   0.3   0.3  0.30                       100.0
13   0.1   0.1   0.1   0.1   0.1  0.10                       1

R2-2-1 to R2-2-5 

Parameters:
- 3.prompt = f"Analyze the sentiment of this article towards {bank_name}. Score the sentiment as either 1 for positive, -1 for negative, or 0 for neutral. Provide the sentiment score (1,0, or -1), followed by a brief explanation in less than 20 words, separating them with a period. Example: '[insert sentiment score]. [insert explanation]'."
    - discrete scaling 1, 0, -1

- temperature = 0, top_p = 0 
- model = gpt-4-turbo-2024-04-09
- non-anonymized data


In [11]:

# Assuming the files are named run1.csv, run2.csv, etc.
file_names_22 = ['Output_Testdata/CS/sample_data_20_20240512_180005_R2-2-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_180005_R2-2-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_180323_R2-2-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_180432_R2-2-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_180522_R2-2-5.csv']
dataframes_22 = [pd.read_csv(file_name) for file_name in file_names_22]

In [12]:
def extract_score(sentiment_text):
    match = re.match(r"(-?\d\.?\d*)\.", sentiment_text)
    return float(match.group(1)) if match else None

for df_22 in dataframes_22:
    df_22['Extracted_Score'] = df_22['Sentiment'].apply(extract_score)

In [13]:
# Create a DataFrame to hold the extracted scores from all runs
scores_22 = pd.DataFrame({
    f'Run{i+1}': df_22['Extracted_Score'] for i, df_22 in enumerate(dataframes_22)
})

# Calculate mean and standard deviation across runs for each article
scores_22['Mean'] = scores_22.mean(axis=1)
#scores['StdDev'] = scores.std(axis=1)

# Function to calculate the percentage of identical values only considering the first 5 columns (run results)
def identical_percentage(row):
    # Isolate the first five columns which are the run results
    data = row[:5]
    unique_values = set(data.dropna())  # Get unique values excluding NaN
    if len(unique_values) == 1:
        return 100.0  # All values are identical
    else:
        # Count occurrences of the most common value
        most_common = max(set(data), key=list(data).count)
        count_most_common = list(data).count(most_common)
        return (count_most_common / len(data.dropna())) * 100

# Apply the function to each row and store the results in a new column
scores_22['Identical Score Percentage'] = scores_22.iloc[:, :5].apply(identical_percentage, axis=1)

# Show the first 20 rows to inspect mean, standard deviation, and identical score percentage
print(scores_22.head(20))

    Run1  Run2  Run3  Run4  Run5  Mean  Identical Score Percentage
0    1.0   1.0   1.0   1.0   1.0   1.0                       100.0
1   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
2    0.0   0.0   0.0   0.0   0.0   0.0                       100.0
3   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
4   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
5   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
6    1.0   1.0   1.0   1.0   1.0   1.0                       100.0
7   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
8   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
9   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
10  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
11  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
12  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
13  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       1

Next i try and figure out a good configuration for temperature and/or top_p. Since gpt-4-turbo is 20x more expensive than gpt-3.5-turbo i will only test the gpt-3.5-turbo model. 

R3-1-1 to R3-1-5 

Parameters:
- 1.prompt = f"Consider the sentiment of this article towards {bank_name}. If {bank_name} is not mentioned, assign a neutral score of 0.5. Otherwise, score the sentiment from 0 to 1, where 0 is very negative and 1 is very positive. Provide the sentiment score followed by a brief explanation in less than 20 words, separating them with a period. Example: '[insert sentiment score]. [insert explanation]'."
    - scaling from 0 to 1 continuous in 0.1 steps
- temperature = 0, top_p = 1 
- model = gpt-3.5-turbo-0125
- non-anonymized data

In [54]:
def extract_score(sentiment_text):
    match = re.match(r"(-?\d\.?\d*)\.", sentiment_text)
    return float(match.group(1)) if match else None

# Function to calculate the percentage of identical values only considering the first 5 columns (run results)
def identical_percentage(row):
    # Isolate the first five columns which are the run results
    data = row[:5]
    unique_values = set(data.dropna())  # Get unique values excluding NaN
    if len(unique_values) == 1:
        return 100.0  # All values are identical
    else:
        # Count occurrences of the most common value
        most_common = max(set(data), key=list(data).count)
        count_most_common = list(data).count(most_common)
        return (count_most_common / len(data.dropna())) * 100
    
    
# Assuming the files are named run1.csv, run2.csv, etc.
file_names_31 = ['Output_Testdata/CS/sample_data_20_20240512_202901_R3-1-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_203037_R3-1-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_203114_R3-1-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_203232_R3-1-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_203319_R3-1-5.csv']
file_names_32 = ['Output_Testdata/CS/sample_data_20_20240512_203456_R3-2-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_203622_R3-2-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_203718_R3-2-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_203814_R3-2-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_203925_R3-2-5.csv']
file_names_33 = ['Output_Testdata/CS/sample_data_20_20240512_204052_R3-3-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_204251_R3-3-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_204612_R3-3-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_204754_R3-3-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_204912_R3-3-5.csv']
file_names_34 = ['Output_Testdata/CS/sample_data_20_20240512_205205_R3-4-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_205320_R3-4-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_205414_R3-4-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_205519_R3-4-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_205614_R3-4-5.csv']
file_names_35 = ['Output_Testdata/CS/sample_data_20_20240512_205856_R3-5-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_210041_R3-5-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_210120_R3-5-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_210215_R3-5-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_210312_R3-5-5.csv']
file_names_36 = ['Output_Testdata/CS/sample_data_20_20240512_210509_R3-6-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_210624_R3-6-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_210715_R3-6-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_210820_R3-6-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_210916_R3-6-5.csv']
file_names_37 = ['Output_Testdata/CS/sample_data_20_20240512_211111_R3-7-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_211236_R3-7-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_211344_R3-7-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_211504_R3-7-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_211630_R3-7-5.csv']
file_names_38 = ['Output_Testdata/CS/sample_data_20_20240512_211804_R3-8-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_212418_R3-8-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_212516_R3-8-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_212615_R3-8-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_212759_R3-8-5.csv']


dataframes_31 = [pd.read_csv(file_name) for file_name in file_names_31]
dataframes_32 = [pd.read_csv(file_name) for file_name in file_names_32]
dataframes_33 = [pd.read_csv(file_name) for file_name in file_names_33]
dataframes_34 = [pd.read_csv(file_name) for file_name in file_names_34]
dataframes_35 = [pd.read_csv(file_name) for file_name in file_names_35]
dataframes_36 = [pd.read_csv(file_name) for file_name in file_names_36]
dataframes_37 = [pd.read_csv(file_name) for file_name in file_names_37]
dataframes_38 = [pd.read_csv(file_name) for file_name in file_names_38]

for df_31 in dataframes_31:
    df_31['Extracted_Score'] = df_31['Sentiment'].apply(extract_score)
for df_32 in dataframes_32:
    df_32['Extracted_Score'] = df_32['Sentiment'].apply(extract_score)
for df_33 in dataframes_33:
    df_33['Extracted_Score'] = df_33['Sentiment'].apply(extract_score)
for df_34 in dataframes_34:
    df_34['Extracted_Score'] = df_34['Sentiment'].apply(extract_score)
for df_35 in dataframes_35:
    df_35['Extracted_Score'] = df_35['Sentiment'].apply(extract_score)
for df_36 in dataframes_36:
    df_36['Extracted_Score'] = df_36['Sentiment'].apply(extract_score)
for df_37 in dataframes_37:
    df_37['Extracted_Score'] = df_37['Sentiment'].apply(extract_score)
for df_38 in dataframes_38:
    df_38['Extracted_Score'] = df_38['Sentiment'].apply(extract_score)

# Create a DataFrame to hold the extracted scores from all runs
scores_31 = pd.DataFrame({
    f'Run{i+1}': df_31['Extracted_Score'] for i, df_31 in enumerate(dataframes_31)
})
scores_32 = pd.DataFrame({
    f'Run{i+1}': df_32['Extracted_Score'] for i, df_32 in enumerate(dataframes_32)
})
scores_33 = pd.DataFrame({
    f'Run{i+1}': df_33['Extracted_Score'] for i, df_33 in enumerate(dataframes_33)
})
scores_34 = pd.DataFrame({
    f'Run{i+1}': df_34['Extracted_Score'] for i, df_34 in enumerate(dataframes_34)
})
scores_35 = pd.DataFrame({
    f'Run{i+1}': df_35['Extracted_Score'] for i, df_35 in enumerate(dataframes_35)
})
scores_36 = pd.DataFrame({
    f'Run{i+1}': df_36['Extracted_Score'] for i, df_36 in enumerate(dataframes_36)
})
scores_37 = pd.DataFrame({
    f'Run{i+1}': df_37['Extracted_Score'] for i, df_37 in enumerate(dataframes_37)
})
scores_38 = pd.DataFrame({
    f'Run{i+1}': df_38['Extracted_Score'] for i, df_38 in enumerate(dataframes_38)
})

# Calculate mean and standard deviation across runs for each article
scores_31['Mean'] = scores_31.mean(axis=1)
#scores['StdDev'] = scores.std(axis=1)
scores_32['Mean'] = scores_32.mean(axis=1)
scores_33['Mean'] = scores_33.mean(axis=1)
scores_34['Mean'] = scores_34.mean(axis=1)
scores_35['Mean'] = scores_35.mean(axis=1)
scores_36['Mean'] = scores_36.mean(axis=1)
scores_37['Mean'] = scores_37.mean(axis=1)
scores_38['Mean'] = scores_38.mean(axis=1)

# Apply the function to each row and store the results in a new column
scores_31['Identical Score Percentage'] = scores_31.iloc[:, :5].apply(identical_percentage, axis=1)
scores_32['Identical Score Percentage'] = scores_32.iloc[:, :5].apply(identical_percentage, axis=1)
scores_33['Identical Score Percentage'] = scores_33.iloc[:, :5].apply(identical_percentage, axis=1)
scores_34['Identical Score Percentage'] = scores_34.iloc[:, :5].apply(identical_percentage, axis=1)
scores_35['Identical Score Percentage'] = scores_35.iloc[:, :5].apply(identical_percentage, axis=1)
scores_36['Identical Score Percentage'] = scores_36.iloc[:, :5].apply(identical_percentage, axis=1)
scores_37['Identical Score Percentage'] = scores_37.iloc[:, :5].apply(identical_percentage, axis=1)
scores_38['Identical Score Percentage'] = scores_38.iloc[:, :5].apply(identical_percentage, axis=1)


# Show the first 20 rows to inspect mean, standard deviation, and identical score percentage
print("top_p = 1, temperature = 0")
print(scores_31.head(20)) 
print("top_p = 1, temperature = 0.3")
print(scores_32.head(20)) 
print("top_p = 1, temperature = 0.6")
print(scores_33.head(20)) 
print("top_p = 1, temperature = 0.9")
print(scores_34.head(20)) 
print("top_p = 0, temperature = 1")
print(scores_35.head(20)) 
print("top_p = 0.3, temperature = 0")
print(scores_36.head(20)) 
print("top_p = 0.6, temperature = 0")
print(scores_37.head(20)) 
print("top_p = 0.9, temperature = 0")
print(scores_38.head(20)) 


top_p = 1, temperature = 0
    Run1  Run2  Run3  Run4  Run5  Mean  Identical Score Percentage
0    0.7   0.7   0.7   0.7   0.7  0.70                       100.0
1    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
2    0.7   0.7   0.7   0.7   0.7  0.70                       100.0
3    0.7   0.7   0.7   0.7   0.7  0.70                       100.0
4    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
5    0.5   0.5   0.5   0.5   0.5  0.50                       100.0
6    0.8   0.8   0.8   0.8   0.8  0.80                       100.0
7    0.3   0.3   0.3   0.3   0.3  0.30                       100.0
8    0.6   0.6   0.6   0.4   0.6  0.56                        80.0
9    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
10   0.7   0.7   0.7   0.7   0.7  0.70                       100.0
11   0.3   0.3   0.2   0.3   0.3  0.28                        80.0
12   0.3   0.3   0.3   0.3   0.3  0.30                       100.0
13   0.2   0.2   0.2   0.2   0.2  0

So as expected, the higher the values for top_p and temperature the less deterministic the model becomes. Anything above 0.3 doesnt make sense for this task. Even 0.3 seems too high. The values dont change much so the sentiment is still scored the same way. So the quality is about the same, but the consistency is worse. 
I think sticking with top_p = 0 and temperature = 0 is the best choice for this task.

In [18]:
print(scores_11.head(20))
print(scores_31.head(20))

    Run1  Run2  Run3  Run4  Run5  Mean  Identical Score Percentage
0    0.7   0.7   0.7   0.7   0.7  0.70                       100.0
1    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
2    0.7   0.7   0.7   0.7   0.7  0.70                       100.0
3    0.7   0.7   0.6   0.6   0.7  0.66                        60.0
4    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
5    0.5   0.5   0.5   0.5   0.5  0.50                       100.0
6    0.8   0.8   0.8   0.8   0.8  0.80                       100.0
7    0.3   0.3   0.3   0.3   0.3  0.30                       100.0
8    0.6   0.6   0.6   0.6   0.6  0.60                       100.0
9    0.2   0.2   0.2   0.2   0.2  0.20                       100.0
10   0.7   0.7   0.7   0.7   0.7  0.70                       100.0
11   0.3   0.3   0.3   0.3   0.3  0.30                       100.0
12   0.3   0.3   0.3   0.3   0.3  0.30                       100.0
13   0.2   0.2   0.2   0.2   0.2  0.20                       1

comparing R1-1-1 (topp = 0,temp = 0) with R3-1-1 (topp=1, temp=0) 

Setting topp to 1 seems to improve consistency slightly, but they are pretty much the same.




So the best parameters so far in terms of consistency are: 

prompt: 
- 1st and 3rd prompt

topp and temperature: 
- either both 0 or temperature = 0 and topp = 1 (because its recommended to only adjust one of the parameters at a time)
- model: gpt-4-turbo-2024-04-09

And in terms of quality the best prompt is the 3rd prompt combined with the gpt-4-turbo-2024-04-09 model. Although the 1st prompt with gpt-4-turbo-2024-04-09 is only slightly worse in terms of quality but gives more detailed scores. 

Next ill test the following: 

Test R4-1-1 to R4-1-5:
Prompt3, topp = 1, temperature = 0, gpt 3.5

Test R4-2-1 to R4-2-5:
Prompt3, topp = 1, temperature = 0, gpt 4

Test R4-3-1 to R4-3-5:
Prompt1, topp = 1, temperature = 0, gpt 4


In [23]:
def extract_score(sentiment_text):
    match = re.match(r"(-?\d\.?\d*)\.", sentiment_text)
    return float(match.group(1)) if match else None

# Function to calculate the percentage of identical values only considering the first 5 columns (run results)
def identical_percentage(row):
    # Isolate the first five columns which are the run results
    data = row[:5]
    unique_values = set(data.dropna())  # Get unique values excluding NaN
    if len(unique_values) == 1:
        return 100.0  # All values are identical
    else:
        # Count occurrences of the most common value
        most_common = max(set(data), key=list(data).count)
        count_most_common = list(data).count(most_common)
        return (count_most_common / len(data.dropna())) * 100
    
    
# Assuming the files are named run1.csv, run2.csv, etc.
file_names_41 = ['Output_Testdata/CS/sample_data_20_20240512_231921_R4-1-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_232018_R4-1-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_232158_R4-1-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_232315_R4-1-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_232419_R4-1-5.csv']
file_names_42 = ['Output_Testdata/CS/sample_data_20_20240512_232900_R4-2-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_233045_R4-2-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_233228_R4-2-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_233324_R4-2-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_233420_R4-2-5.csv']
file_names_43 = ['Output_Testdata/CS/sample_data_20_20240512_233702_R4-3-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_233816_R4-3-2.csv',
              'Output_Testdata/CS/sample_data_20_20240512_233920_R4-3-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_234049_R4-3-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_234257_R4-3-5.csv']


dataframes_41 = [pd.read_csv(file_name) for file_name in file_names_41]
dataframes_42 = [pd.read_csv(file_name) for file_name in file_names_42]
dataframes_43 = [pd.read_csv(file_name) for file_name in file_names_43]


for df_41 in dataframes_41:
    df_41['Extracted_Score'] = df_41['Sentiment'].apply(extract_score)
for df_42 in dataframes_42:
    df_42['Extracted_Score'] = df_42['Sentiment'].apply(extract_score)
for df_43 in dataframes_43:
    df_43['Extracted_Score'] = df_43['Sentiment'].apply(extract_score)

# Create a DataFrame to hold the extracted scores from all runs
scores_41 = pd.DataFrame({
    f'Run{i+1}': df_41['Extracted_Score'] for i, df_41 in enumerate(dataframes_41)
})
scores_42 = pd.DataFrame({
    f'Run{i+1}': df_42['Extracted_Score'] for i, df_42 in enumerate(dataframes_42)
})
scores_43 = pd.DataFrame({
    f'Run{i+1}': df_43['Extracted_Score'] for i, df_43 in enumerate(dataframes_43)
})

# Calculate mean and standard deviation across runs for each article
scores_41['Mean'] = scores_41.mean(axis=1)
#scores['StdDev'] = scores.std(axis=1)
scores_42['Mean'] = scores_42.mean(axis=1)
scores_43['Mean'] = scores_43.mean(axis=1)


# Apply the function to each row and store the results in a new column
scores_41['Identical Score Percentage'] = scores_41.iloc[:, :5].apply(identical_percentage, axis=1)
scores_42['Identical Score Percentage'] = scores_42.iloc[:, :5].apply(identical_percentage, axis=1)
scores_43['Identical Score Percentage'] = scores_43.iloc[:, :5].apply(identical_percentage, axis=1)


# Show the first 20 rows to inspect mean, standard deviation, and identical score percentage
print("3rd prompt, top_p = 1, temperature = 0")
print(scores_41.head(20)) 
print("3rd prompt, top_p = 1, temperature = 0")
print(scores_42.head(20)) 
print("1st prompt, top_p = 1, temperature = 0")
print(scores_43.head(20)) 


3rd prompt, top_p = 1, temperature = 0
    Run1  Run2  Run3  Run4  Run5  Mean  Identical Score Percentage
0   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
1   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
2   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
3   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
4   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
5   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
6    0.0   0.0   0.0   0.0   0.0   0.0                       100.0
7   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
8   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
9   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
10   0.0   0.0   0.0   0.0   0.0   0.0                       100.0
11  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
12  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
13  -1.0  -1.0  -1.0  -

setting topp to and temperature to 0 or setting both to 0 seems to result in almost the same consistency. I think i will follow the recommendation of only adjusting one parameter at a time and go with temperature = 0 and topp = 1. This might also have a positive effect on the explanation part of the output. So the top model would be: 3rd prompt, topp = 1, temperature = 0, gpt 4 turbo. Now i want to find out if anonymizing the data has an effect on the scores. 


Test R5-1-1 to R5-1-5:
Prompt3, top = 1, temperature = 0, gpt 3.5, anonymized data
Test R5-2-1 to R5-2-5:
Prompt3, top = 1, temperature = 0, gpt 4, anonymized data


In [21]:

# Assuming the files are named run1.csv, run2.csv, etc.
file_names = ['Output_Testdata/CS/anonymized_data_20_20240513_001258_R5-1-1.csv',
              'Output_Testdata/CS/anonymized_data_20_20240513_001657_R5-1-2.csv',
              'Output_Testdata/CS/anonymized_data_20_20240513_001820_R5-1-3.csv',
              'Output_Testdata/CS/anonymized_data_20_20240513_001920_R5-1-4.csv',
              'Output_Testdata/CS/anonymized_data_20_20240513_002024_R5-1-5.csv']
dataframes = [pd.read_csv(file_name) for file_name in file_names]

def extract_score(sentiment_text):
    match = re.match(r"(-?\d\.?\d*)\.", sentiment_text)
    return float(match.group(1)) if match else None

for df in dataframes:
    df['Extracted_Score'] = df['Sentiment'].apply(extract_score)
    

# Create a DataFrame to hold the extracted scores from all runs
scores = pd.DataFrame({
    f'Run{i+1}': df['Extracted_Score'] for i, df in enumerate(dataframes)
})

# Calculate mean and standard deviation across runs for each article
scores['Mean'] = scores.mean(axis=1)
#scores['StdDev'] = scores.std(axis=1)

# Function to calculate the percentage of identical values only considering the first 5 columns (run results)
def identical_percentage(row):
    # Isolate the first five columns which are the run results
    data = row[:5]
    unique_values = set(data.dropna())  # Get unique values excluding NaN
    if len(unique_values) == 1:
        return 100.0  # All values are identical
    else:
        # Count occurrences of the most common value
        most_common = max(set(data), key=list(data).count)
        count_most_common = list(data).count(most_common)
        return (count_most_common / len(data.dropna())) * 100

# Apply the function to each row and store the results in a new column
scores['Identical Score Percentage'] = scores.iloc[:, :5].apply(identical_percentage, axis=1)

# Show the first 20 rows to inspect mean, standard deviation, and identical score percentage
print(scores.head(20))

    Run1  Run2  Run3  Run4  Run5  Mean  Identical Score Percentage
0   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
1   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
2   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
3   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
4   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
5   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
6    0.0   0.0   0.0   0.0   0.0   0.0                       100.0
7   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
8   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
9   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
10   0.0   0.0   0.0  -1.0   0.0  -0.2                        80.0
11  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
12  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
13  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       1

In [39]:

# Assuming the files are named run1.csv, run2.csv, etc.
file_names_52 = ['Output_Testdata/CS/anonymized_data_20_20240513_002442_R5-2-1.csv',
              'Output_Testdata/CS/anonymized_data_20_20240513_002700_R5-2-2.csv',
              'Output_Testdata/CS/anonymized_data_20_20240513_002829_R5-2-3.csv',
              'Output_Testdata/CS/anonymized_data_20_20240513_002925_R5-2-4.csv',
              'Output_Testdata/CS/anonymized_data_20_20240513_003019_R5-2-5.csv']
dataframes_52 = [pd.read_csv(file_name) for file_name in file_names_52]

def extract_score(sentiment_text):
    match = re.match(r"(-?\d\.?\d*)\.", sentiment_text)
    return float(match.group(1)) if match else None

for df_52 in dataframes_52:
    df_52['Extracted_Score'] = df_52['Sentiment'].apply(extract_score)
    

# Create a DataFrame to hold the extracted scores from all runs
scores_52 = pd.DataFrame({
    f'Run{i+1}': df_52['Extracted_Score'] for i, df_52 in enumerate(dataframes_52)
})

# Calculate mean and standard deviation across runs for each article
scores_52['Mean'] = scores_52.mean(axis=1)
#scores['StdDev'] = scores.std(axis=1)

# Function to calculate the percentage of identical values only considering the first 5 columns (run results)
def identical_percentage(row):
    # Isolate the first five columns which are the run results
    data = row[:5]
    unique_values = set(data.dropna())  # Get unique values excluding NaN
    if len(unique_values) == 1:
        return 100.0  # All values are identical
    else:
        # Count occurrences of the most common value
        most_common = max(set(data), key=list(data).count)
        count_most_common = list(data).count(most_common)
        return (count_most_common / len(data.dropna())) * 100

# Apply the function to each row and store the results in a new column
scores_52['Identical Score Percentage'] = scores_52.iloc[:, :5].apply(identical_percentage, axis=1)

# Show the first 20 rows to inspect mean, standard deviation, and identical score percentage
print(scores_52.head(20))

    Run1  Run2  Run3  Run4  Run5  Mean  Identical Score Percentage
0    1.0   1.0   1.0   1.0   1.0   1.0                       100.0
1   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
2    0.0   0.0   0.0   0.0   0.0   0.0                       100.0
3   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
4   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
5   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
6    1.0   1.0   1.0   1.0   1.0   1.0                       100.0
7   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
8   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
9   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
10  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
11  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
12  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
13  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       1

# Choosing the right model: 

## Consistency
### R1 Tests: 
parameters:
    prompt 1 vs 2 vs 3,
        with gpt-3.5-turbo, top_p = 0, temp = 0, non-anonymized data

R1 showed that prompt 1 and prompt 3 performed best. Prompt 1 was inconsistent with 2 articles and prompt 3 with 3 articles. 

### R2 Tests:
parameters:
    prompt 1 vs 3,
        with gpt-4-turbo, top_p = 0, temp = 0, non-anonymized data

R2 showed that prompt 1 was inconsistent in only 1 article and prompt 3 was fully consistent. 

### R3 Tests: 
parameters:
    top_p=1, temp = 0 vs
    top_p=1, temp = 0.3 vs
    top_p=1, temp = 0.6 vs
    top_p=1, temp = 0.9 vs
    top_p=0, temp = 1 vs
    top_p=0.3, temp = 0 vs
    top_p=0.6, temp = 0 vs
    top_p=0.9, temp = 0 vs
        with prompt 1, gpt-3.5-turbo, non-anonymized data, 

R3 showed that top_p = 1 and temp = 0 was the most consistent with 2 articles being inconsistent (so same as with top_p = 0 and temperature = 0)

### R4 Tests:
parameters:
    prompt 3, top_p = 1, temp = 0, gpt-3.5-turbo, non-anonymized data
    -> inconsistent with 2 articles (with top_p = 0 and temp = 0 it was inconsistent with 3 articles)
    prompt 3, top_p = 1, temp = 0, gpt-4-turbo, non-anonymized data
    -> fully consistent (same as with top_p = 0 and temp = 0)
    prompt 1, top_p = 1, temp = 0, gpt-4-turbo, non-anonymized data
    -> inconsistent with 2 articles (with top_p = 0 and temp = 0 it was inconsistent with 1 article)

### R5 Tests:
parameters:
    prompt 3, top_p = 1, temp = 0, gpt-3.5-turbo, anonymized data
    -> inconsistent with 4 articles (with non-anonymized data it was inconsistent with 2 articles)
    prompt 3, top_p = 1, temp = 0, gpt-4-turbo, anonymized data
    -> fully consistent (same as with non-anonymized data)

R5 showed that, with the gpt-3.5 model, anonymizing data had a negative effect on consistency. With the gpt-4 model, anonymizing data had no effect on consistency.

### Conclusion:
Top models are: 
- R4-2: prompt 3, top_p = 1, temp = 0, gpt-4-turbo, non-anonymized data
    -> fully consistent
- R5-2: prompt 3, top_p = 1, temp = 0, gpt-4-turbo, anonymized data
    -> fully consistent
- R2-2: prompt 3, top_p = 0, temp = 0, gpt-4-turbo, non-anonymized data
    -> fully consistent
- R2-1: prompt 1, top_p = 0, temp = 0, gpt-4-turbo, non-anonymized data
    -> inconsistent with 1 article
- R4-3: prompt 1, top_p = 1, temp = 0, gpt-4-turbo, non-anonymized data
    -> inconsistent with 2 articles
- R1-1: prompt 1, top_p = 0, temp = 0, gpt-3.5-turbo, non-anonymized data
    -> inconsistent with 2 articles
- R3-1: prompt 1, top_p = 0, temp = 1, gpt-3.5-turbo, non-anonymized data
    -> inconsistent with 2 articles
- R4-1: prompt 3, top_p = 1, temp = 0, gpt-3.5-turbo, non-anonymized data
    -> inconsistent with 2 articles



## Quality of sentiment evaluation

### Personal evaluation of articles:
Article 0: Probably neutral to positiv. Some positive sentiment (returns up), CS generally in a difficult position. 

Article 1: Very negative (Archegos stuff)

Article 2: Neutral – CS providing insights 

Article 3: Negative – spying scandal

Article 4: Eher negative (AT1 bonds after takeover)

Article 5: Eher negative (article is mainly about deutsche bank not being the next credit Suisse)

Article 6: (Neutral,) slightly positive (CS helping Climate bonds initiative)

Article 7: Negative – lawsuits blabla

Article 8: Neutral to slightly negative. (princeling hiring practice in banking sector)

Article 9: Negative (credit Suisse takeover, restructuring blabla…) 

Article 10: More or less neutral, negativ towards previous management, optimistic about new management. Could be anything honestly. 

Article 11: Eher negativ

Article 12: Eher negativ

Article 13: Negativ.

Article 14: Neutral to slightly positiv. 

Article 15: Negativ 

Article 16: Slightly negativ to neutral

Article 17: Negativ

Article 18: Optimistic, probably neutral to slightly positiv

Article 19: Handelt von der PUK – neutral to slightly negativ


### Evaluation Procedure: 

If the gpt evaluation is consistent with my evaluation, ill give a point. If it is off completely ill give 0 points. Some articles are hard to evaluate because they have both negative and positive aspects. 

### R4-2, R5-2, R2-2 have the same scores

0. good
1. good
2. good
3. good
4. good
5. good
6. good
7. good
8. good
9. good
10. good
11. good
12. good
13. good
14. good
15. good
16. good
17. good
18. good
19. good

--> 20 points

In [46]:
new_df = pd.concat([scores_42["Mean"], scores_52["Mean"], scores_22["Mean"]], axis=1)

# Optionally, setting new column names
new_df.columns = ['R42', 'R52', 'R22']

new_df

,R42,R52,R22
0,1.0,1.0,1.0
1,-1.0,-1.0,-1.0
2,0.0,0.0,0.0
3,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0
5,-1.0,-1.0,-1.0
6,1.0,1.0,1.0
7,-1.0,-1.0,-1.0
8,-1.0,-1.0,-1.0
9,-1.0,-1.0,-1.0


### R2-1 and R4-3 have the same scores

0. good
1. good
2. bad
3. bad
4. good
5. good
6. good
7. good
8. good
9. good
10. good
11. good
12. good
13. good
14. good
15. good
16. good
17. good
18. good
19. good

-> 18 points

In [52]:
new_df = pd.concat([scores_21["Mean"], scores_43["Mean"]], axis=1)

# Optionally, setting new column names
new_df.columns = ['R21', 'R43']

new_df

,R21,R43
0,0.70,0.70
1,0.20,0.20
2,0.70,0.70
3,0.40,0.38
4,0.20,0.20
5,0.14,0.10
6,0.80,0.80
7,0.20,0.20
8,0.20,0.20
9,0.20,0.20


In [ ]:

dataframes = []
for i, file_name in enumerate(file_names):
    df = pd.read_csv(file_name)
    # Assume 'Sentiment' is the column to be extracted
    df = df[['Sentiment']].rename(columns={'Sentiment': f'Run{i+1}'})
    dataframes.append(df)

### R1-1 and R3-1 have the same scores

0. good
1. good
2. bad
3. bad
4. good
5. bad
6. good
7. good
8. ok
9. good
10. good
11. good
12. good
13. good
14. good
15. good
16. bad
17. good
18. good
19. good

-> 16 points


In [55]:
new_df = pd.concat([scores_11["Mean"], scores_31["Mean"]], axis=1)

# Optionally, setting new column names
new_df.columns = ['R11', 'R31']

new_df

,R11,R31
0,0.70,0.70
1,0.20,0.20
2,0.70,0.70
3,0.66,0.70
4,0.20,0.20
5,0.50,0.50
6,0.80,0.80
7,0.30,0.30
8,0.60,0.56
9,0.20,0.20


### R41

0. bad
1. good
2. bad
3. good
4. good
5. good
6. good
7. good
8. good
9. good
10. good
11. good
12. good
13. good
14. bad
15. good
16. good
17. good
18. good
19. good

--> 17 points

In [57]:
scores_41["Mean"]

0    -1.0
1    -1.0
2    -1.0
3    -1.0
4    -1.0
5    -1.0
6     0.0
7    -1.0
8    -1.0
9    -1.0
10    0.0
11   -1.0
12   -1.0
13   -1.0
14   -0.8
15   -1.0
16    0.0
17   -1.0
18    0.0
19   -0.6
Name: Mean, dtype: float64

## Conclusion:

Prompt3 with gpt-4-turbo, top_p = 1, temp = 0 is the best model for this task. Anonymizing data has no effect on the quality of the sentiment evaluation. It outperforms the other models in terms of consistency and quality.

Could also take top_p = 0 and temp = 0 but it is not recommended and since it doesnt make a difference in this case, i will go with top_p = 1 and temp = 0.

... könnte noch schauen ob top_p = 0 und temp = 0 vs top_p = 1 und temp = 0 unterschiedlich bezüglich der erklärung sind. Aber ist mir eigentlich schnurz. Hab die erklärung ursprünglich einfach zur kontrolle gebraucht. 

#### Tests R6: 


In [2]:
# Assuming the files are named run1.csv, run2.csv, etc.
file_names_61 = ['Output_Testdata/CS/sample_data_20_20240514_093529_R6-1-1.csv',
              'Output_Testdata/CS/sample_data_20_20240514_093625_R6-1-2.csv',
              'Output_Testdata/CS/sample_data_20_20240514_093724_R6-1-3.csv',
              'Output_Testdata/CS/sample_data_20_20240514_093817_R6-1-4.csv',
              'Output_Testdata/CS/sample_data_20_20240514_093914_R6-1-5.csv']
dataframes_61 = [pd.read_csv(file_name) for file_name in file_names_61]

def extract_score(sentiment_text):
    match = re.match(r"(-?\d\.?\d*)\.", sentiment_text)
    return float(match.group(1)) if match else None

for df_61 in dataframes_61:
    df_61['Extracted_Score'] = df_61['Sentiment'].apply(extract_score)
    

# Create a DataFrame to hold the extracted scores from all runs
scores_61 = pd.DataFrame({
    f'Run{i+1}': df_61['Extracted_Score'] for i, df_61 in enumerate(dataframes_61)
})

# Calculate mean and standard deviation across runs for each article
scores_61['Mean'] = scores_61.mean(axis=1)
#scores['StdDev'] = scores.std(axis=1)

# Function to calculate the percentage of identical values only considering the first 5 columns (run results)
def identical_percentage(row):
    # Isolate the first five columns which are the run results
    data = row[:5]
    unique_values = set(data.dropna())  # Get unique values excluding NaN
    if len(unique_values) == 1:
        return 100.0  # All values are identical
    else:
        # Count occurrences of the most common value
        most_common = max(set(data), key=list(data).count)
        count_most_common = list(data).count(most_common)
        return (count_most_common / len(data.dropna())) * 100

# Apply the function to each row and store the results in a new column
scores_61['Identical Score Percentage'] = scores_61.iloc[:, :5].apply(identical_percentage, axis=1)

# Show the first 20 rows to inspect mean, standard deviation, and identical score percentage
print(scores_61.head(20))

    Run1  Run2  Run3  Run4  Run5  Mean  Identical Score Percentage
0    1.0   1.0   1.0   1.0   1.0   1.0                       100.0
1   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
2    0.0   0.0   0.0   0.0   0.0   0.0                       100.0
3    0.0   0.0   0.0   0.0   0.0   0.0                       100.0
4   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
5   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
6    1.0   1.0   1.0   1.0   1.0   1.0                       100.0
7   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
8    0.0   0.0   0.0   0.0   0.0   0.0                       100.0
9   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
10   0.0   0.0   0.0   0.0   0.0   0.0                       100.0
11  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
12  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
13  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       1

# R6-2


In [3]:
# Assuming the files are named run1.csv, run2.csv, etc.
file_names_62 = ['Output_Testdata/CS/sample_data_20_20240514_095936_R6-2-1.csv',
              'Output_Testdata/CS/sample_data_20_20240514_100229_R6-2-2.csv',
              'Output_Testdata/CS/sample_data_20_20240514_100322_R6-2-3.csv',
              'Output_Testdata/CS/sample_data_20_20240514_100422_R6-2-4.csv',
              'Output_Testdata/CS/sample_data_20_20240514_100520_R6-2-5.csv']
dataframes_62 = [pd.read_csv(file_name) for file_name in file_names_62]

def extract_score(sentiment_text):
    match = re.match(r"(-?\d\.?\d*)\.", sentiment_text)
    return float(match.group(1)) if match else None

for df_62 in dataframes_62:
    df_62['Extracted_Score'] = df_62['Sentiment'].apply(extract_score)
    

# Create a DataFrame to hold the extracted scores from all runs
scores_62 = pd.DataFrame({
    f'Run{i+1}': df_62['Extracted_Score'] for i, df_62 in enumerate(dataframes_62)
})

# Calculate mean and standard deviation across runs for each article
scores_62['Mean'] = scores_62.mean(axis=1)
#scores['StdDev'] = scores.std(axis=1)

# Function to calculate the percentage of identical values only considering the first 5 columns (run results)
def identical_percentage(row):
    # Isolate the first five columns which are the run results
    data = row[:5]
    unique_values = set(data.dropna())  # Get unique values excluding NaN
    if len(unique_values) == 1:
        return 100.0  # All values are identical
    else:
        # Count occurrences of the most common value
        most_common = max(set(data), key=list(data).count)
        count_most_common = list(data).count(most_common)
        return (count_most_common / len(data.dropna())) * 100

# Apply the function to each row and store the results in a new column
scores_62['Identical Score Percentage'] = scores_62.iloc[:, :5].apply(identical_percentage, axis=1)

# Show the first 20 rows to inspect mean, standard deviation, and identical score percentage
print(scores_62.head(20))

    Run1  Run2  Run3  Run4  Run5  Mean  Identical Score Percentage
0    1.0   1.0   1.0   1.0   1.0   1.0                       100.0
1   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
2    0.0   0.0   0.0   0.0   0.0   0.0                       100.0
3    0.0   0.0   0.0   0.0   0.0   0.0                       100.0
4   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
5   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
6    1.0   1.0   1.0   1.0   1.0   1.0                       100.0
7   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
8    0.0   0.0   0.0   0.0   0.0   0.0                       100.0
9   -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
10   0.0   0.0   0.0   0.0   0.0   0.0                       100.0
11  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
12  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       100.0
13  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0                       1

#### R2-2 vs R6-2

In [5]:
file_names = ['Output_Testdata/CS/sample_data_20_20240512_180005_R2-2-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_180005_R2-2-1.csv',
              'Output_Testdata/CS/sample_data_20_20240512_180323_R2-2-3.csv',
              'Output_Testdata/CS/sample_data_20_20240512_180432_R2-2-4.csv',
              'Output_Testdata/CS/sample_data_20_20240512_180522_R2-2-5.csv']

dataframes = []
for i, file_name in enumerate(file_names):
    df = pd.read_csv(file_name)
    # Assume 'Sentiment' is the column to be extracted
    df = df[['Sentiment']].rename(columns={'Sentiment': f'Run{i+1}'})
    dataframes.append(df)

In [7]:
dataframes[0]

,Run1
0,1. Positive earnings report and strategic rest...
1,-1. Article highlights failures in risk manage...
2,0. The article neutrally presents Credit Suiss...
3,-1. Focus on leadership turmoil and spying sca...
4,-1. Article highlights significant losses and ...
5,-1. Article highlights Credit Suisse's trouble...
6,1. Credit Suisse is portrayed positively for p...
7,-1. Article highlights Credit Suisse's legal s...
8,-1. Article mentions Credit Suisse's involveme...
9,"-1. Article highlights ongoing struggles, inve..."


In [8]:
file_names = ['Output_Testdata/CS/sample_data_20_20240514_095936_R6-2-1.csv',
              'Output_Testdata/CS/sample_data_20_20240514_100229_R6-2-2.csv',
              'Output_Testdata/CS/sample_data_20_20240514_100322_R6-2-3.csv',
              'Output_Testdata/CS/sample_data_20_20240514_100422_R6-2-4.csv',
              'Output_Testdata/CS/sample_data_20_20240514_100520_R6-2-5.csv']

dataframes = []
for i, file_name in enumerate(file_names):
    df = pd.read_csv(file_name)
    # Assume 'Sentiment' is the column to be extracted
    df = df[['Sentiment']].rename(columns={'Sentiment': f'Run{i+1}'})
    dataframes.append(df)

In [9]:
dataframes[1]

,Run2
0,1. Positive financial performance and strategi...
1,-1. The article highlights significant failure...
2,0. The article presents Credit Suisse's analys...
3,0. Mixed reactions to leadership changes and s...
4,-1. The article highlights significant losses ...
5,-1. The article highlights Credit Suisse's for...
6,1. Credit Suisse is portrayed positively for i...
7,-1. The article highlights ongoing legal troub...
8,0. Credit Suisse is mentioned factually withou...
9,"-1. The article highlights ongoing struggles, ..."
